In [1]:
# mvp_pi1/
# │
# ├── datos/                              # Datos crudos y transformados
# │   ├── crudos_raw/                     # Datos originales (CSV, JSON, etc.)
# │   └── transformados_processed/        # Datos limpios después del ETL
# │
# ├── notebooks/                # Notebooks Jupyter para análisis y desarrollo exploratorio
# │   └── eda_notebook.ipynb    # Análisis exploratorio de datos (EDA)
# │
# ├── src/                      # Código fuente del proyecto
# │   ├── __init__.py           # Archivo para definir la carpeta como un módulo Python
# │   ├── etl.py                # Código para transformaciones (ETL)
# │   ├── api.py                # Código para la API con FastAPI
# │   ├── recommendation.py     # Sistema de recomendación
# │   └── validation.py         # Validación de datos (con la clase o funciones de validación)
# │
# ├── tests/                    # Pruebas unitarias
# │   └── test_api.py           # Pruebas para la API
# │
# ├── requirements.txt          # Dependencias del proyecto (FastAPI, pandas, scikit-learn, etc.)
# ├── README.md                 # Documentación general del proyecto
# └── main.py                   # Punto de entrada del proyecto (ejecución principal)


In [2]:
# 1 - Importamos librerías a utilizar durante el MVP
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import json as js 
import ast  
import re 
import sys
import os

sys.path.append('/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1')
import src.etl as etl 

import src.etl as etl
print(dir( etl ))

# conectar la carpera src (/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/src) contenida en la carpeta del proyecto mvp_pi1 (/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1) que contiene a etl.py y api.py
sys.path.insert(1, '/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/src')
import etl as etl # 
import api as api # 


# 2 - Cargamos los dataset tipo csv a través de pandas
movies = pd.read_csv('/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/crudos_raw/movies_dataset.csv', low_memory=False)
credits = pd.read_csv('/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/crudos_raw/credits.csv')
df_movies = movies
df_credits = credits


# 3 - Exploramos visualmente las primeras 3 filas de cada dataset para entender su estructura y contenido
pd.set_option('display.width', 1000)  # Establece un ancho suficientemente grande para la impresion de columnas
pd.set_option('display.expand_frame_repr', False)  # Evita que se divida el DataFrame en varias líneas en la impresion
pd.set_option('display.max_colwidth', None) # Establece máximo ancho posible para evitar truncamiento de texto en la impresion
print(df_movies.head(4))
print(df_credits.head(4))



# 4.1 - Exploramos los tipos de datos y caracteristicas del dataset movies referente a cantidad de columnas, 
# cantidad de datos, tipo de datos, nulos, duplicados, ceros, inconsistentes, vacios y negativos.
# Importar la función "validar_df" desde el archivo etl.py
from src.etl import validar_df
import importlib
importlib.reload(etl)

# Ajustar Pandas para que no corte la impresión en varias líneas
pd.set_option('display.expand_frame_repr', False)  # Muestra la tabla en una sola línea horizontal
pd.set_option('display.max_columns', None)         # Asegura que se muestren todas las columnas sin recortarlas
pd.set_option('display.width', 1000)               # Ajusta el ancho máximo permitido para la impresión
print(validar_df(df_movies)) # Imprimir la información del dataframe movies_dataset.csv



# 4.2 - Exploramos los tipos de datos y caracteristicas del dataset "credits" referente a cantidad de columnas, 
# cantidad de datos, tipo de datos, nulos, duplicados, ceros, inconsistentes, vacios y negativos.
# Importar la función "validar_df" desde el archivo etl.py
from src.etl import validar_df
import importlib
importlib.reload(etl)

# Ajustar Pandas para que no corte la salida en varias líneas
pd.set_option('display.expand_frame_repr', False)  # Muestra la tabla en una sola línea si la pantalla es ancha
pd.set_option('display.max_columns', None)         # Asegura que se muestren todas las columnas sin truncarlas
pd.set_option('display.width', 1000)               # Ajusta el ancho máximo permitido para la salida
print(validar_df(df_credits)) # Imprimir la información del dataframe credits.csv



# En la exploración de los dataset se identifica los dos comparten un id unico comun asociado a cada pelicula,
# adicionalmente partiendo de los requerimientos del proyecto se empiezan a tener en cuenta tipos de datos a setiar,
# columnas a transformar, eliminar, renombrar, crear, tambien datos a transformar y eliminar, y al final una depuración
# de datos y dataframes a unificar para la creación de un nuevo dataset que permita consulta a traves de una API y 
# la creación de un sistema de recomendación.

# 5 - Partiedo de tener los diccionario con sus tipos de datos correspondientes procedemos a dar formato homogeneo
# a los datos de cada columna previendo futuros usos a algunas columnas en particular
diccionario_tipos_movies = {
    'adult': bool,  # Debe ser un valor booleano (True o False)
    'belongs_to_collection': dict,  # Un diccionario con detalles de la colección a la que pertenece la película
    'budget': int,  # Presupuesto de la película, debe ser un número entero
    'genres': list,  # Lista de géneros en formato de diccionarios
    'homepage': str,  # Página web oficial de la película
    'id': int,  # ID único de la película
    'imdb_id': str,  # ID de la película en IMDb
    'original_language': str,  # Idioma original de la película
    'original_title': str,  # Título original de la película
    'overview': str,  # Resumen o descripción de la película
    'popularity': float,  # Puntaje de popularidad, un número decimal
    'poster_path': str,  # Ruta al póster de la película
    'production_companies': list,  # Lista de compañías productoras (diccionarios)
    'production_countries': list,  # Lista de países de producción (diccionarios)
    'release_date': pd.Timestamp,  # Fecha de estreno de la película
    'revenue': int,  # Recaudación total de la película
    'runtime': float,  # Duración de la película en minutos
    'spoken_languages': list,  # Lista de idiomas hablados en la película
    'status': str,  # Estado de la película (por ejemplo, "Released")
    'tagline': str,  # Frase célebre o eslogan de la película
    'title': str,  # Título de la película
    'video': bool,  # Indica si hay un video disponible
    'vote_average': float,  # Promedio de votos recibidos
    'vote_count': float  # Número total de votos
}
diccionario_tipos_credits = {
    'cast': list,  # Lista de diccionarios que representa los actores del reparto de la película.
    'crew': list,  # Una lista de diccionarios que representa los miembros del equipo de producción de la película.
    'id': int,  # ID único de la película
}



# 5.1 - Convertimos los tipos de datos de las columnas del dataset movies_dataset.csv según el diccionario de tipos
# Importa las funciones necesarias desde el archivo etl.py
from src.etl import convertir_tipos # Función para convertir los tipos de datos segun un diccionario dado
import importlib
importlib.reload(etl)

# Aplicamos la conversión de tipos al dataframe movies_dataset.csv
df_movies = convertir_tipos(df_movies, diccionario_tipos_movies) 
# Validamos como quedó ahora el dataset luego de aplicar los formatos indicados con el diccionario de tipos
print(validar_df(df_movies)) # Imprimir la información del dataframe movies_dataset.csv



# 5.2 - Convertimos los tipos de datos de las columnas del dataset credits.csv según el diccionario de tipos
# Importa las funciones necesarias desde el archivo etl.py
from src.etl import convertir_tipos # Función para convertir los tipos de datos segun un diccionario dado
import importlib
importlib.reload(etl)

# Aplicamos la conversión de tipos al dataframe credits.csv
df_credits = convertir_tipos(df_credits, diccionario_tipos_credits) 
# Validamos como quedó ahora el dataset luego de aplicar los formatos indicados con el diccionario de tipos
print(validar_df(df_credits)) # Imprimir la información del dataframe credits.csv



# 6.1 - Validamos la estructura de las filas del df_movies segun el diccionario de tipos
from src.etl import validar_estructura_df
# Validamos la estructura a nivel de filas del dataframe movies según el diccionario y sus tipos 
# de datos y posteriormente borramos las filas inconsistentes
df_movies = df_movies.drop(validar_estructura_df(df_movies, diccionario_tipos_movies))
# se actualizan los indices del df_movies luego de borra las filas incosnistentes
df_movies = df_movies.reset_index(drop=True) 
print(validar_estructura_df(df_movies, diccionario_tipos_movies))
# Validamos como quedaron las columnas y filas del dataset luego de aplicar las transformaciones
print(validar_df(df_movies))



# 6.2 - Validamos la estructura de las filas del df credits segun el diccionario de tipos
from src.etl import validar_estructura_df
# Validamos la estructura a nivel de filas del dataframe credits según el diccionario y sus tipos 
# de datos y posteriormente borramos las filas inconsistentes encontradas
df_credits = df_credits.drop(validar_estructura_df(df_credits, diccionario_tipos_credits))
# se actualizan los indices del df credits luego de borrar las filas incosnistentes
df_credits = df_credits.reset_index(drop=True) 
print(validar_estructura_df(df_credits, diccionario_tipos_credits))
# Validamos como quedaron las columnas y filas del dataset luego de aplicar las transformaciones
print(validar_df(df_credits))



### (Requerimiento de Aprobación) ###
# Ahora se crea la columna release_year en el df movies
df_movies['release_year'] = df_movies['release_date'].dt.year # Extrae el año de la fecha de estreno
df_movies['release_year'] = df_movies['release_year'].astype('Int64') # Convierte a tipo entero
# Eliminamos los valores nulos en la columna release_date y actualizamos los indices
df_movies = df_movies.dropna(subset=['release_date']).reset_index(drop=True) 
print(validar_df(df_movies)) # Imprimir la información del dataframe movies



### (Requerimiento de Aprobación) ###
# Eliminar las columnas que no serán utilizadas: video,imdb_id,adult,original_title,poster_path y homepage.
print(df_movies.columns)
# Ahora se borran las columnas: video,imdb_id,adult,original_title,poster_path y homepage solo si existen
df_movies = df_movies.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'], errors='ignore')
# Reorganizamos las columnas, ponemos id de primeras y acercamos las columnas revenue con budget, y release_year con release_date 
df_movies = df_movies.reindex(columns=['id','title','tagline','release_date','release_year','runtime','budget','revenue','genres','production_companies','production_countries','spoken_languages','original_language','overview','popularity','vote_average','vote_count','belongs_to_collection','status'])
print(df_movies.columns)



### (Requerimiento de Aprobación) ###
# Crear la columna con el retorno de inversión llamada return con la division de los campos campos revenue y budget
# cuando no haya datos disponibles para calcularlo deberá tomar el valor 0, incluyendo la division en cero.
# Aseguramos que 'revenue' y 'budget' sean numéricos; en caso de error se convierten a NaN y se reemplazan por 0
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce').fillna(0)
df_movies['budget']  = pd.to_numeric(df_movies['budget'], errors='coerce').fillna(0)
# Calcular el retorno de inversión
df_movies['return'] = df_movies['revenue'] / df_movies['budget']
# Reemplazar divisiones por cero o resultados infinitos por 0, y llenar posibles NaN
df_movies['return'] = df_movies['return'].replace([np.inf, -np.inf], 0).fillna(0)
# Asegurarse de que el tipo es float y redondear a 2 decimales
df_movies['return'] = df_movies['return'].astype(float).round(2)
#print(validar_df(df_movies)) # Imprimir la información del dataframe movies_dataset.csv


# Renombramos la columna 'id' a 'movie_id' del df_movies
df_movies.rename(columns={'id': 'movie_id'}, inplace=True) # Renombra la columna 'id' a 'movie_id'
# cambia a la primera posicion la columna movie_id
columnas = ['movie_id'] + [col for col in df_movies.columns if col != 'movie_id'] # Columnas en el orden deseado
df_movies = df_movies[columnas] # Reordena las columnas
print(validar_df(df_movies)) # Imprimir la información del dataframe movies_dataset.csv



# 7 Ahora trabajamos el desanidado de las columnas 'cast' y 'crew' del dataset credits.csv
# Renombramos la columna 'id' a 'movie_id'
df_credits.rename(columns={'id': 'movie_id'}, inplace=True) # Renombra la columna 'id' a 'movie_id'
# cambia a la primera posicion la columna movie_id
columnas = ['movie_id'] + [col for col in df_credits.columns if col != 'movie_id'] # Columnas en el orden deseado
df_credits = df_credits[columnas] # Reordena las columnas
print(validar_df(df_credits)) # Imprimir la información del dataframe credits.csv



# 7.1 - Obtenemos el listado de los campos contenidos en los diccionarios de las columnas 'cast' y 'crew'
# Importar la función "obtener_campos_json" desde el archivo etl.py
from src.etl import obtener_campos_json
from src.etl import extraer_campos_json
import importlib
importlib.reload(etl)

# Se obtienen los campos presentes en los diccionarios de la columna 'cast'
campos_cast = obtener_campos_json(df_credits,'cast')
campos_cast.sort() # Ordena alfabéticamente
campos_crew = obtener_campos_json(df_credits,'crew')
campos_crew.sort() # Ordena alfabéticamente
print(campos_cast) # Imprime la lista de campos presentes en 'cast'
print(campos_crew) # Imprime la lista de campos presentes en 'crew'

df_cast = extraer_campos_json(df_credits,'cast',campos_cast)
df_crew = extraer_campos_json(df_credits,'crew',campos_crew)

# cambia a la primera posicion la columna movie_id
columnas = ['movie_id'] + [col for col in df_crew.columns if col != 'movie_id'] # Columnas en el orden deseado
df_crew = df_crew[columnas] # Reordena las columnas
columnas = ['movie_id'] + [col for col in df_cast.columns if col != 'movie_id'] # Columnas en el orden deseado
df_cast = df_cast[columnas] # Reordena las columnas

print(validar_df(df_cast)) 
print(validar_df(df_crew))

# Finalmente generamos los tres dataset a partir de los tres df procesados (df_movies, df_crew, df_cast)
df_movies.to_csv(os.path.join("/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/transformados_processed", "data_movies.csv"), index=False)
df_cast.to_csv(os.path.join("/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/transformados_processed", "data_cast.csv"), index=False)
df_crew.to_csv(os.path.join("/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/transformados_processed", "data_crew.csv"), index=False)

# Guardar el DataFrame en formato Parquet
df_movies.to_parquet(path="/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/transformados_processed/data_movies.parquet", index=False)
df_cast.to_parquet(path="/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/transformados_processed/data_cast.parquet", index=False)
df_crew.to_parquet(path="/Users/usuario/Documents/DATA_SCIENCE/M7_LABs_Proyectos_individuales/mvp_pi1/transformados_processed/data_crew.parquet", index=False)

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'ast', 'convertir_a_estructura', 'convertir_tipos', 'cosine_similarity', 'datetime', 'extraer_campo', 'extraer_campos_json', 'formato_fecha', 'np', 'obtener_campos_json', 'pd', 'validar_df', 'validar_estructura_csv', 'validar_estructura_df', 'validar_tipo']
   adult                                                                                                                                        belongs_to_collection    budget                                                                                             genres                              homepage     id    imdb_id original_language     original_title                                                                                                                                                                                                                                                                            